In [1]:
import requests
from pprint import pprint
#https://realpython.com/python-requests/

In [2]:
CMR_OPS = 'https://cmr.earthdata.nasa.gov/search'
url = f'{CMR_OPS}/{"collections"}'

In [15]:
#990 datasets
# pare down to PODAAC only
provider = 'POCLOUD'
response = requests.get(url,
                        params={
                            'cloud_hosted': 'True',
                            'has_granules': 'True',
                            'provider': provider,
                            'page_size': 251,

                        },
                        headers={
                            'Accept': 'application/json'
                        }
                       )

In [16]:
response.headers['CMR-Hits']

'251'

In [17]:
collections = response.json()['feed']['entry']

In [22]:
#for collection in collections:
#    print(f'{collection["archive_center"]} {collection["dataset_id"]} {collection["id"]}')

In [21]:
collections = response.json()['feed']['entry']
for collection in collections:
    print(f'{collection["dataset_id"]} {collection["id"]} {collection["short_name"]} ')
    #if "SST" in ID:
    #print(ID)

Sentinel-6A MF Jason-CS L2 P4 Altimeter Low Resolution (LR) NRT Ocean Surface Topography C1968979597-POCLOUD JASON_CS_S6A_L2_ALT_LR_STD_OST_NRT_F 
GHRSST Level 4 MUR Global Foundation Sea Surface Temperature Analysis (v4.1) C1996881146-POCLOUD MUR-JPL-L4-GLOB-v4.1 
Sentinel-6A MF Jason-CS L2 P4 Altimeter Low Resolution (LR) STC Ocean Surface Topography C1968980609-POCLOUD JASON_CS_S6A_L2_ALT_LR_STD_OST_STC_F 
MetOp-A ASCAT Level 2 25.0 km Ocean Surface Wind Vectors C2075141524-POCLOUD ASCATA-L2-25km 
ECCO Ocean Temperature and Salinity - Monthly Mean 0.5 Degree (Version 4 Release 4) C1990404795-POCLOUD ECCO_L4_TEMP_SALINITY_05DEG_MONTHLY_V4R4 
Sentinel-6A MF Jason-CS L2 P4 Altimeter Low Resolution (LR) NTC Ocean Surface Topography (Unvalidated) C1968979551-POCLOUD JASON_CS_S6A_L2_ALT_LR_STD_OST_NTC_F_UNVALIDATED 
Northeast Weddell Sea Pre-SWOT Level-4 Hourly MITgcm LLC4320 Native Grid 2km Oceanographic Dataset Version 1.0 C2006849345-POCLOUD MITgcm_LLC4320_Pre-SWOT_JPL_L4_ROAM_MIZ_v1.0

In [48]:
response = requests.get(url, 
                        params={
                            'concept_id': 'C1968979551-POCLOUD',
                            'temporal': '2018-10-17T00:00:00Z,2018-10-18T23:59:59Z',
                            'bounding_box': '76.08166,-67.1746,88.19689,21.04862',
                            'page_size': 200,
                            },
                        headers={
                            'Accept': 'application/json'
                            }
                       )
print(response.status_code)
# confine to bounding box, time range, and certain dataset

200


In [49]:
response.headers['CMR-Hits']

'0'

In [32]:
granules = response.json()['feed']['entry']
for granule in granules:
    print(f'{granule["dataset_id"]} {granule["id"]}')

MEaSUREs Gridded Sea Surface Height Anomalies Version 1812 C2036880672-POCLOUD


In [33]:
pprint(granules[0])

{'archive_center': 'NASA/JPL/PODAAC',
 'associations': {'services': ['S2004184019-POCLOUD'],
                  'tools': ['TL2108419875-POCLOUD'],
                  'variables': ['V2112016831-POCLOUD',
                                'V2112016837-POCLOUD',
                                'V2112016842-POCLOUD',
                                'V2112016847-POCLOUD',
                                'V2112016852-POCLOUD',
                                'V2146305584-POCLOUD',
                                'V2146305586-POCLOUD',
                                'V2146305590-POCLOUD']},
 'boxes': ['-90 -180 90 180'],
 'browse_flag': True,
 'coordinate_system': 'CARTESIAN',
 'data_center': 'POCLOUD',
 'dataset_id': 'MEaSUREs Gridded Sea Surface Height Anomalies Version 1812',
 'has_formats': True,
 'has_spatial_subsetting': True,
 'has_temporal_subsetting': True,
 'has_transforms': False,
 'has_variables': True,
 'id': 'C2036880672-POCLOUD',
 'links': [{'href': 'https://podaac-tools.jpl.nasa.

In [34]:
len(granules)

1

In [45]:

glist = []
print(type(glist))
for gr in granules:
    glist.append(gr['links'][0]['href'])
    print(glist[-1])
print(len(glist))

<class 'list'>
https://podaac-tools.jpl.nasa.gov/drive/files/allData/merged_alt/L4/docs/alti-gridding-jpl-PODAAC-UserGuide_20200227.pdf
1


In [50]:
import requests
from pprint import pprint
from pathlib import Path

import s3fs

import xarray as xr

import matplotlib.pyplot as plt
import cartopy.crs as ccrs